In [2]:
_predict_df = "select (Version.[Version Name]*Product.[Product].[208821]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource,PluginSetting

# register inputs
predict_df = O9DataLake.register("predict_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _predict_df,plugin_setting = PluginSetting.Inputs)
systemreport = O9DataLake.register("sysreport",data_source = DataSource.LIVEFRAME,entity_type = ResourceType.LIVEFRAME,plugin_setting = PluginSetting.Inputs)
liveinput = O9DataLake.register("ReportName",data_source = DataSource.LIVEFRAME,entity_type = ResourceType.LIVEFRAME,plugin_setting = PluginSetting.Inputs)

# register slice dimension
O9DataLake.register("Product.[Product]", data_source = DataSource.LS, entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.SliceDimension)

# register outputs
O9DataLake.register("output1",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)
O9DataLake.register("output2",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

# register script params
script_params = O9DataLake.register({"var1":"20","var2":"Test1","var3":"newparam"}, data_source = DataSource.LS, plugin_setting = PluginSetting.ScriptParam)

In [3]:
O9DataLake.register("Version.[Version Name]", data_source = DataSource.LS, entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.SliceDimension)

In [4]:
O9DataLake.inputs

{'predict_df': {'name': 'predict_df',
  'resource_type': <ResourceType.IBPL: 'ibpl_query'>,
  'data_source': <DataSource.LS: 'liveserver'>,
  'query': 'select (Version.[Version Name]*Product.[Product].[208821]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});',
  'std_count_limit': '200000',
  'df':     Version.[Version Name]  Product.[Product] Time.[FiscalWeek]  \
  0                       S1             208821          W49-2015   
  1       CurrentWorkingView             208821          W49-2015   
  2                       S1             208821          W50-2015   
  3       CurrentWorkingView             208821          W50-2015   
  4                       S1             208821          W51-2015   
  ..                     ...                ...               ...   
  315     CurrentWorkingView             208821          W50-201

In [5]:
# fetching inputs
predict_df = O9DataLake.get('predict_df')
systemreport = O9DataLake.get('sysreport')
liveinput = O9DataLake.get('ReportName')
# fetching script params
value1 = O9DataLake.get_script_param("var1")
value2 = O9DataLake.get_script_param("var2")

In [6]:
#user script

# package imports
import logging
from sklearn import tree

# initialize output variables
output1 = None
output2 = None


# initialize logger
logger = logging.getLogger('o9_logger')

logger.debug(f'predict_df dataframe:  {predict_df.shape}')
logger.debug(f'systemreport dataframe:  {systemreport.shape}')
logger.debug(f'liveinput dataframe:  {liveinput.shape}')
logger.debug(f'script param var1 value: {value1}')
logger.debug(f'script param var2 value: {value2}')

2024-06-26 13:40:44,192 - o9_logger - DEBUG  - predict_df dataframe:  (320, 10)
2024-06-26 13:40:44,193 - o9_logger - DEBUG  - systemreport dataframe:  (421570, 9)
2024-06-26 13:40:44,193 - o9_logger - DEBUG  - liveinput dataframe:  (421570, 4)
2024-06-26 13:40:44,194 - o9_logger - DEBUG  - script param var1 value: 20
2024-06-26 13:40:44,195 - o9_logger - DEBUG  - script param var2 value: Test1


In [11]:
output1 = predict_df
logger.debug(f'systemreport dataframe:  {systemreport.head()}')

2024-06-26 13:43:21,488 - o9_logger - DEBUG  - systemreport dataframe:    WalmartTime.[Day]  Store.[Store_ID]  Department.[Department_ID]  \
0        02-05-2010                 1                           1   
1        02-05-2010                 1                           2   
2        02-05-2010                 1                           3   
3        02-05-2010                 1                           4   
4        02-05-2010                 1                           5   

   DSML dsmldim         CPI  Unemployment  Fuel Price  Weekly Sales  \
0           NaN  211.096358         8.106       2.572      24924.50   
1           NaN  211.096358         8.106       2.572      50605.27   
2           NaN  211.096358         8.106       2.572      13740.12   
3           NaN  211.096358         8.106       2.572      39954.04   
4           NaN  211.096358         8.106       2.572      32229.38   

   Predicted Sales  
0     18810.473617  
1     37932.229054  
2     14496.019182  
3 

In [8]:
output2 = liveinput
logger.debug(f'systemreport dataframe:  {liveinput.head()}')

2024-06-26 13:40:52,131 - o9_logger - DEBUG  - systemreport dataframe:    WalmartTime.[Day]  Department.[Department_ID]  Store.[Store_ID]  \
0        02-05-2010                           1                 1   
1        02-05-2010                           1                10   
2        02-05-2010                           1                11   
3        02-05-2010                           1                12   
4        02-05-2010                           1                13   

   Weekly Sales  
0      24924.50  
1      40212.84  
2      19611.13  
3      17426.75  
4      46761.90  


In [12]:
#pushing outputs to Live Server(make sure all dimensions,measures present in LiveServer)
O9DataLake.put('output1', output1)


In [10]:
O9DataLake.put('output2', output2)

2024-06-26 19:12:23.934719: LiveServer : Failed to parse file. Cannot parse the header row.
2024-06-26 19:12:23.934776: LiveServer : Failed to parse file. Cannot parse the header row.
2024-06-26 19:12:23.934781: LiveServer : Failed to parse file. Cannot parse the header row.
2024-06-26 19:12:23.934784: LiveServer : Header:[System.String[]]
2024-06-26 19:12:23.934787: LiveServer : Failed to parse file. Cannot parse the header row.
2024-06-26 19:12:23.934790: LiveServer : Header:[System.String[]]
2024-06-26 19:12:23.934793: LiveServer : Measure [DSML dsmldim] is not setup for spreading
2024-06-26 19:12:23.934796: LiveServer : Measure [DSML dsmldim] is not setup for spreading
2024-06-26 19:12:23.934799: LiveServer : Header:[System.String[]]
2024-06-26 19:12:23.934802: LiveServer : Measure [DSML dsmldim] is not setup for spreading
2024-06-26 19:12:23.934805: LiveServer : Header:[System.String[]]
2024-06-26 19:12:23.934808: LiveServer : Measure [DSML dsmldim] is not setup for spreading
2024